In [ ]:
import cv2
import numpy as np

# Initialize webcam
cap = cv2.VideoCapture(0)

# Define skin color range in HSV
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame for a mirror effect
    frame = cv2.flip(frame, 1)

    # Define Region of Interest
    roi = frame[100:400, 100:400]

    # Convert ROI to HSV and apply skin color mask
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Apply Gaussian blur and morphological operations for noise reduction
    skin_mask = cv2.GaussianBlur(skin_mask, (5, 5), 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

    # Find contours from the skin mask
    contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour and apply area threshold
        max_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(max_contour) > 2000: 
            # Draw the contour and convex hull on the ROI
            cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)
            hull = cv2.convexHull(max_contour, returnPoints=False)

            if hull is not None and len(hull) > 3:
                defects = cv2.convexityDefects(max_contour, hull)
                if defects is not None:
                    thumb_up = False
                    thumb_down = False
                    thumb_tip = None
                    deepest_defect = None

                    max_depth = 0
                    for i in range(defects.shape[0]):
                        start_idx, end_idx, far_idx, depth = defects[i, 0]
                        start = tuple(max_contour[start_idx][0])
                        end = tuple(max_contour[end_idx][0])
                        far = tuple(max_contour[far_idx][0])

                        # Draw the start, end, and far points for defects
                        cv2.circle(roi, start, 5, (0, 255, 0), -1)
                        cv2.circle(roi, end, 5, (0, 0, 255), -1)
                        cv2.circle(roi, far, 5, (255, 0, 0), -1)
                        cv2.line(roi, start, end, (0, 255, 255), 2)

                        # Consider the deepest defect as the thumb point
                        if depth > max_depth:
                            max_depth = depth
                            thumb_tip = start if start[1] < end[1] else end
                            deepest_defect = far

                    # Check if the thumb_tip is above or below hand center to classify gesture
                    if thumb_tip and deepest_defect:
                        x, y, w, h = cv2.boundingRect(max_contour)
                        hand_center_y = y + h // 2  # Center Y of bounding box

                        if thumb_tip[1] < hand_center_y:  # Thumb is above center
                            thumb_up = True
                        elif thumb_tip[1] > hand_center_y:  # Thumb is below center
                            thumb_down = True

                    # Display the gesture result
                    if thumb_up:
                        cv2.putText(frame, "Thumbs Up", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    elif thumb_down:
                        cv2.putText(frame, "Thumbs Down", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the skin mask and detected hand contour
    cv2.imshow("Skin Mask", skin_mask)
    cv2.imshow("Hand Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 